In [ ]:
from os import path

# Third-party
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
from gala.dynamics.mockstream import fardal_stream
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
pot = gp.MilkyWayPotential()
H = gp.Hamiltonian(pot)

In [ ]:
e_factors = np.array([1., 0.84, 0.63, 0.515, 0.09])

# for ef in e_factors:
#     w0 = gd.PhaseSpacePosition(pos=[10, 0, 0]*u.kpc,
#                                vel=[0, 0, 234.2*ef]*u.km/u.s)
#     orbit = pot.integrate_orbit(w0, dt=0.5, t1=0, t2=6*u.Gyr)
    # print(orbit.eccentricity())

In [ ]:
Ms = [1e4, 1e5, 1e6, 1e7, 1e8] * u.Msun
streams = []
i = 0
for ef in e_factors:
    for m in Ms:
        cache_filename = '../cache/{0}.hdf5'.format(i)
        if path.exists(cache_filename):
            continue
            
        w0 = gd.PhaseSpacePosition(pos=[10, 0, 0]*u.kpc,
                                   vel=[0, 0, 234.2*ef]*u.km/u.s)
        orbit = pot.integrate_orbit(w0, dt=0.5, t1=0, t2=4*u.Gyr)
        
        print('dude', m)
        stream = fardal_stream(H, orbit, m, release_every=8, 
                               snapshot_filename=cache_filename)
        i += 1
    break

In [ ]:
i = 0
cache_filename = '../cache/{0}.hdf5'.format(i)

with h5py.File(cache_filename, 'r') as f:
    print(f['pos'].shape)
    
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

In [ ]:
2000 * 8001 * 6 * 8 / 1024 / 1024 / 1024